In [ ]:
!pip install transformers

     |████████████████████████████████| 890kB 9.0MB/s 
     |████████████████████████████████| 3.0MB 5.5MB/s 
     |████████████████████████████████| 1.1MB 54.0MB/s 
     |████████████████████████████████| 890kB 55.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=182f74da4bcf259e0400ceec3693e679b632f53d7506571b6e050200c9312e2e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
# Importing the libs

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tarfile
from sklearn.model_selection import train_test_split
import json
from sklearn.preprocessing import MultiLabelBinarizer
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download("stopwords")
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))

import transformers
import numpy as np
from transformers import TFXLNetForSequenceClassification, XLNetTokenizer

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# Downloading the dataset

url = 'http://www.cs.cmu.edu/~ark/personas/data/MovieSummaries.tar.gz'
dataset = tf.keras.utils.get_file('dataset', url)

48005120/48002242 [==============================] - 78s 2us/step


In [ ]:
# Extracting the dataset

tarred_file = tarfile.open(dataset)
tarred_file.extractall(path="/data")
tarred_file.close()

In [ ]:
ls /data/MovieSummaries/

character.metadata.tsv  name.clusters.txt   README.txt
movie.metadata.tsv      plot_summaries.txt  tvtropes.clusters.txt


In [ ]:
# !cat /data/MovieSummaries/README.txt

In [ ]:
# Reading the movie metadata

meta_cols = ['W_ID', 'F_ID', 'Name', 'Release_Date', 'Revenue', 'Runtime', 'Languages', 'Countries', 'Genres']

movie_meta = pd.read_csv('/data/MovieSummaries/movie.metadata.tsv', delimiter='\t', header=None, names=meta_cols)

In [ ]:
movie_meta.head()

,W_ID,F_ID,Name,Release_Date,Revenue,Runtime,Languages,Countries,Genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


In [ ]:
movie_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81741 entries, 0 to 81740
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   W_ID          81741 non-null  int64  
 1   F_ID          81741 non-null  object 
 2   Name          81741 non-null  object 
 3   Release_Date  74839 non-null  object 
 4   Revenue       8401 non-null   float64
 5   Runtime       61291 non-null  float64
 6   Languages     81741 non-null  object 
 7   Countries     81741 non-null  object 
 8   Genres        81741 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 5.6+ MB


In [ ]:
# Reading movie summaries

plot_cols = ['W_ID', 'Plot']
plot_sum = pd.read_csv("/data/MovieSummaries/plot_summaries.txt", sep='\t', encoding='utf-8', header=None, names=plot_cols)
plot_sum.head()

,W_ID,Plot
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...


In [ ]:
plot_sum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42303 entries, 0 to 42302
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   W_ID    42303 non-null  int64 
 1   Plot    42303 non-null  object
dtypes: int64(1), object(1)
memory usage: 661.1+ KB


In [ ]:
# Merging movie metadata and movie summaries

result = pd.merge(plot_sum, movie_meta[['W_ID', 'Genres']], on = 'W_ID')
result.head(10)

,W_ID,Plot,Genres
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha...","{""/m/07s9rl0"": ""Drama"", ""/m/03q4nz"": ""World ci..."
1,31186339,The nation of Panem consists of a wealthy Capi...,"{""/m/03btsm8"": ""Action/Adventure"", ""/m/06n90"":..."
2,20663735,Poovalli Induchoodan is sentenced for six yea...,"{""/m/04t36"": ""Musical"", ""/m/02kdv5l"": ""Action""..."
3,2231378,"The Lemon Drop Kid , a New York City swindler,...","{""/m/06qm3"": ""Screwball comedy"", ""/m/01z4y"": ""..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
5,5272176,The president is on his way to give a speech. ...,"{""/m/01jfsb"": ""Thriller"", ""/m/03btsm8"": ""Actio..."
6,1952976,"{{plot}} The film opens in 1974, as a young gi...","{""/m/01jfsb"": ""Thriller"", ""/m/07s9rl0"": ""Drama..."
7,24225279,"The story begins with Hannah, a young Jewish t...","{""/m/07s9rl0"": ""Drama"", ""/m/02b5_l"": ""Teen""}"
8,2462689,Infuriated at being told to write one final co...,"{""/m/06cvj"": ""Romantic comedy"", ""/m/0hj3nby"": ..."
9,20532852,A line of people drool at the window of the s...,"{""/m/02hmvc"": ""Short Film"", ""/m/0hqxf"": ""Famil..."


In [ ]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42204 entries, 0 to 42203
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   W_ID    42204 non-null  int64 
 1   Plot    42204 non-null  object
 2   Genres  42204 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


In [ ]:
def extract_values(dic):
  """ Extract the Genre data
      Args:
        dic: key value pair with key containing the actual genre
  """
  val = list(json.loads(dic).values())
  return val

result['Genres'] = result['Genres'].apply(extract_values)

In [ ]:
result.head()

,W_ID,Plot,Genres
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha...","[Drama, World cinema]"
1,31186339,The nation of Panem consists of a wealthy Capi...,"[Action/Adventure, Science Fiction, Action, Dr..."
2,20663735,Poovalli Induchoodan is sentenced for six yea...,"[Musical, Action, Drama, Bollywood]"
3,2231378,"The Lemon Drop Kid , a New York City swindler,...","[Screwball comedy, Comedy]"
4,595909,Seventh-day Adventist Church pastor Michael Ch...,"[Crime Fiction, Drama, Docudrama, World cinema..."


In [ ]:
def pre_process_text(text):
  """ Preprocessing of the text
    Args:
      text: The text to be pre-processed
  """
  text = text.lower()
  text = re.sub("[^a-z]"," ",text)
  text = ' '.join(text.split())
  return text


result['Plot'] = result['Plot'].apply(pre_process_text)

In [ ]:
def remove_stopwords(text):
  """ Removes stopwords from the text
    Args:
      text: The text from which stop words are to be removed
  """
  proc_text = [w for w in text.split() if not w in stop_words]
  return " ".join(proc_text)

In [ ]:
result['Plot'] = result['Plot'].apply(remove_stopwords)

In [ ]:
word_lemmatizer = WordNetLemmatizer()

def lemmatize(text):
  """ Lemmatize the text
    Args:
      text : The text which has to be lemmatized
  """
  proc_text = [word_lemmatizer.lemmatize(w) for w in text.split()]
  return " ".join(proc_text)

In [ ]:
result['Plot'] = result['Plot'].apply(lemmatize)

In [ ]:
# Converting all Genre values in lower case

result['Genres'] = result['Genres'].apply(lambda x: [g.lower() for g in x])

In [ ]:
# Fucntion to append all genre in list. Just used for counting freq of each genre

genre_list = []
for genre in result['Genres']:
  genre_list.extend(genre)


In [ ]:
len(genre_list)

150775

In [ ]:
# Counting the frequency of each genre

from collections import Counter

counts = Counter(genre_list)

In [ ]:
# Creating a dataframe of genre along with its frequency

genre_count_df = pd.DataFrame({
    'Genre': list(counts.keys()),
    'Count': list(counts.values())
})

In [ ]:
print(genre_count_df.sort_values(by=['Count']))

            Genre  Count
362    neorealism      1
323   ninja movie      1
346       c-movie      1
340   linguistics      1
350    breakdance      1
..            ...    ...
4          action   5868
12       thriller   6530
20   romance film   6666
8          comedy  10467
0           drama  19134

[363 rows x 2 columns]


In [ ]:
threshold = 20

In [ ]:
# Creating a dataframe of genre with count less than threshold

less_than_thresh = genre_count_df[genre_count_df['Count'] < threshold]

remove_list = less_than_thresh['Genre'].values

In [ ]:
len(remove_list)

147

In [ ]:
def remove_below_thresh(item):
  """ Removes all genre which have frequency less than the threshold value
  """
  new_item = []
  for genre in item:
    if genre not in remove_list:
      new_item.append(genre)

  return new_item


In [ ]:
# Removing genre below threshold

result['Genres'] = result['Genres'].apply(remove_below_thresh)

In [ ]:
# Counting the no of genre left after removing below threshold genres

new_genlist = []
for genre in result['Genres']:
  new_genlist.extend(genre)

In [ ]:
len(set(new_genlist))

216

In [ ]:
set(new_genlist)

{'absurdism',
 'action',
 'action comedy',
 'action thrillers',
 'action/adventure',
 'addiction drama',
 'adult',
 'adventure',
 'adventure comedy',
 'airplanes and airports',
 'alien film',
 'americana',
 'animal picture',
 'animated cartoon',
 'animated musical',
 'animation',
 'anime',
 'anti-war',
 'anti-war film',
 'apocalyptic and post-apocalyptic fiction',
 'art film',
 'auto racing',
 'avant-garde',
 'b-movie',
 'backstage musical',
 'biker film',
 'biographical film',
 'biography',
 'biopic [feature]',
 'black comedy',
 'black-and-white',
 'blaxploitation',
 'bollywood',
 'boxing',
 'british empire film',
 'buddy film',
 'caper story',
 'chase movie',
 'childhood drama',
 "children's",
 "children's fantasy",
 "children's/family",
 'chinese movies',
 'christian film',
 'christmas movie',
 'combat films',
 'comedy',
 'comedy film',
 'comedy of errors',
 'comedy of manners',
 'comedy thriller',
 'comedy western',
 'comedy-drama',
 'coming of age',
 'computer animation',
 'costum

In [ ]:
# Transformation dictionary to be used for transforming each Genre

transform_dict = {
    'action comedy': ['action', 'comedy'],
    'action thrillers': ['action', 'thriller'],
    'action/adventure': ['action', 'adventure'],
    'addiction drama': ['drama'],
    'adventure comedy': ['adventure', 'comedy'],
    'animated cartoon': ['anime'],
    'animated musical': ['anime', 'musical'],
    'animation': ['anime'],
    'anti-war film': ['anti-war'],
    'auto racing': ['sports'],
    'backstage musical': ['musical'],
    'biographical film': ['biography'],
    'biopic [feature]': ['biography'],
    'crime comedy': ['crime', 'comedy'],
    'crime drama': ['crime', 'drama'],
    'crime fiction': ['crime', 'fiction'],
    'crime thriller': ['crime', 'thriller'],
    'costume adventure': ['costume', 'adventure'],
    'costume drama': ['costume', 'drama'],
    'costume horror': ['costume', 'horror'],
    'childhood drama': ['childhood', 'drama'],
    "children's": ['childhood'],
    "children's fantasy": ['childhood', 'fantasy'],
    "children's/family": ['childhood', 'family'],
    'christian film': ['religious film'],
    'combat films' :['action'],
    'comedy film': ['comedy'],
    'comedy of errors': ['comedy'],
    'comedy of manners': ['comedy'],
    'comedy thriller': ['comedy', 'thriller'],
    'comedy western': ['comedy', 'western'],
    'comedy-drama': ['comedy', 'drama'],
    'computer animation': ['anime'],
    'courtroom drama': ['drama'],
    'cult': ['culture'],
    'culture & society': ['culture', 'social'],
    'detective fiction': ['detective', 'fiction'],
    'docudrama': ['documentary', 'drama'],
    'doomsday film': ['apocalyptic and post-apocalyptic fiction'],
    'domestic comedy': ['comedy'],
    'drame': ['drama'],
     'erotic drama': ['adult', 'drama'],
    'erotic thriller': ['adult', 'thriller'],
    'erotica': ['adult'],
    'family drama': ['family', 'drama'],
    'family film': ['family'],
    'family-oriented adventure': ['family', 'adventure'],
    'fantasy adventure': ['fantasy', 'adventure'],
    'fantasy comedy': ['fantasy', 'comedy'],
    'future noir': ['film noir'],
    'gay interest': ['gay'],
    'gay pornography': ['gay', 'adult'],
    'gay themed': ['gay'],
    'giallo': ['horror', 'thriller'],
    'gothic film': ['horror'],
    'glamorized spy film': ['spy'],
    'gross-out film': ['gross out'],
    'heavenly comedy': ['comedy'],
    'heaven-can-wait fantasies': ['fantasy'],
    'historical drama': ['history', 'drama'],
    'historical epic': ['history', 'epic'],
    'historical fiction': ['history', 'fiction'],
    'haunted house film': ['horror'],
    'horror comedy': ['horror', 'comedy'],
    'hybrid western': ['western'],
    'inspirational drama': ['drama'],
    'law & crime': ['law', 'crime'],
     'lgbt': ['gay'],
    'master criminal films': ['crime'],
    'marriage drama': ['drama'],
    'melodrama': ['drama'],
    'medical fiction': ['fiction'],
    'monster movie': ['monster'],
    'mockumentary' : ['documentary', 'comedy'],
    'music': ['musical'],
    'musical comedy': ['musical', 'comedy'],
    'musical drama': ['musical', 'drama'],
    'mythological fantasy': ['fantasy'],
    'natural horror films': ['horror'],
    'parody': ['comedy'],
    'pinku eiga': ['adult'],
    'psychological thriller': ['thriller'],
    'political cinema': ['political'],
    'political drama': ['political', 'drama'],
    'political satire': ['political'],
    'political thriller': ['political', 'thriller'],
    'pornographic movie': ['adult'],
    'revisionist western': ['western'],
    'rockumentary': ['musical', 'documentary'],
    'romance film': ['romance'],
    'romantic comedy': ['romance', 'comedy'],
    'romantic drama': ['romance', 'drama'],
    'romantic fantasy': ['romance', 'fantasy'],
    'sexploitation': ['adult'],
    'softcore porn': ['adult'],
    'spaghetti western': ['western'],
    'sci-fi adventure': ['sci-fi', 'adventure'],
    'sci-fi horror': ['sci-fi', 'horror'],
    'science fiction': ['sci-fi'],
    'slasher': ['horror'],
    'sex comedy': ['adult', 'comedy'],
    'social issues': ['social'],
    'social problem film': ['social'],
    'splatter film': ['horror'],
    'superhero movie': ['superhero'],
    'sword and sorcery films': ['sword and sorcery'],
     'tragicomedy': ['tragedy', 'comedy'],
     'whodunit': ['detective'],
    'workplace comedy': ['comedy'],
     'wuxia': ['martial arts film'],


     'erotic': ['adult'],
     'pornography': ['adult']


}

In [ ]:
# List containing Genre which has to be removed. As they are not actually Genres

to_remove_set = {
    'americana',
    'bollywood',
    'chinese movies',
    'christmas movie',
    'experimental film',
    'filipino movies',
    'indie',
    'japanese movies',
    'new hollywood',
    'remake',
    'roadshow theatrical release',
    'samurai cinema',
    'short film',
    'tollywood',
    'tamil cinema',
    'television movie',
    'world cinema',
}

In [ ]:
def transform_genres(item):
  """ Transforms Genre according the transformation dict. Also removes
      Genres which in to remove list.
      Args:
        item: list of Genres
  """
  transformed_genre_item = []
  for genre in item:
    if genre in transform_dict:
      transformed_genre_item.extend(transform_dict[genre])
    elif genre not in to_remove_set :
      transformed_genre_item.append(genre)

  return list(set(transformed_genre_item))


In [ ]:
# Applies transformation to Genres

result['Genres'] = result['Genres'].apply(transform_genres)

In [ ]:
result.head(20)

,W_ID,Plot,Genres
0,23890098,shlykov a hard working taxi driver and lyosha ...,[drama]
1,31186339,the nation of panem consists of a wealthy capi...,"[sci-fi, adventure, action, drama]"
2,20663735,poovalli induchoodan is sentenced for six year...,"[action, musical, drama]"
3,2231378,the lemon drop kid a new york city swindler is...,"[comedy, screwball comedy]"
4,595909,seventh day adventist church pastor michael ch...,"[fiction, drama, documentary, crime]"
5,5272176,the president is on his way to give a speech w...,"[adventure, action, drama, thriller]"
6,1952976,plot the film opens in as a young girl dahlia ...,"[drama, horror, thriller]"
7,24225279,the story begins with hannah a young jewish te...,"[teen, drama]"
8,2462689,infuriated at being told to write one final co...,"[media satire, drama, black-and-white, romance..."
9,20532852,a line of people drool at the window of the sh...,"[family, anime]"


In [ ]:
# Counting the no of Genres left

new_genlist = []
for genre in result['Genres']:
  new_genlist.extend(genre)

In [ ]:
len(set(new_genlist))

100

In [ ]:
set(new_genlist)

{'absurdism',
 'action',
 'adult',
 'adventure',
 'airplanes and airports',
 'alien film',
 'animal picture',
 'anime',
 'anti-war',
 'apocalyptic and post-apocalyptic fiction',
 'art film',
 'avant-garde',
 'b-movie',
 'biker film',
 'biography',
 'black comedy',
 'black-and-white',
 'blaxploitation',
 'boxing',
 'british empire film',
 'buddy film',
 'caper story',
 'chase movie',
 'childhood',
 'comedy',
 'coming of age',
 'costume',
 'creature film',
 'crime',
 'culture',
 'dance',
 'detective',
 'disaster',
 'documentary',
 'drama',
 'dystopia',
 'ensemble film',
 'epic',
 'escape film',
 'existentialism',
 'fairy tale',
 'family',
 'fantasy',
 'feminist film',
 'fiction',
 'film adaptation',
 'film noir',
 'film à clef',
 'gangster film',
 'gay',
 'gross out',
 'heist',
 'hip hop movies',
 'history',
 'holiday film',
 'horror',
 'interpersonal relationships',
 'jungle film',
 'juvenile delinquency film',
 'law',
 'martial arts film',
 'media satire',
 'monster',
 'musical',
 'mys

In [ ]:
# Printing all those instances which has genre list empty and has to be removed

result[result['Genres'].apply(lambda x: len(x) == 0 )]

,W_ID,Plot,Genres
26,10644072,a villager new to the big city bhagwaan turns ...,[]
39,28238240,mickey and the scorpions decide to go on a big...,[]
43,28207941,bimbo and koko are sign painter s they have be...,[]
58,18392317,betty is startled awake in her bed on a stormy...,[]
138,25807103,tone jane is a secretary who doesn t know that...,[]
...,...,...,...
42119,18253063,speedy gonzales owns the hottest nightclub in ...,[]
42130,30350202,after a nuclear apocalypse the whole landscape...,[]
42146,27376208,the life of the s m theme artist and author se...,[]
42163,4108329,what appears to be a moose emerging from a lak...,[]


In [ ]:
# Removing all those instances which has empty genre list

result = result[result['Genres'].apply(lambda x: len(x) != 0 )]

In [ ]:
# checking if there is any instance with empty genre list

result[result['Genres'].apply(lambda x: len(x) == 0 )]

,W_ID,Plot,Genres


In [ ]:
# Counting the no of Genres

new_genlist = []
for genre in result['Genres']:
  new_genlist.extend(genre)

In [ ]:
len(set(new_genlist))

100

In [ ]:
# Counting the no of words in each movie summary

result['No words'] = result['Plot'].apply(lambda x : len(x.split()))

In [ ]:
result.head()

,W_ID,Plot,Genres,No words
0,23890098,shlykov a hard working taxi driver and lyosha ...,[drama],28
1,31186339,the nation of panem consists of a wealthy capi...,"[sci-fi, adventure, action, drama]",779
2,20663735,poovalli induchoodan is sentenced for six year...,"[action, musical, drama]",493
3,2231378,the lemon drop kid a new york city swindler is...,"[comedy, screwball comedy]",862
4,595909,seventh day adventist church pastor michael ch...,"[fiction, drama, documentary, crime]",406


In [ ]:
# Getting states of no of words in each summary

result['No words'].agg(['min', 'max', 'mean'])

min       13.000000
max     5015.000000
mean     318.640777
Name: No words, dtype: float64

In [ ]:
# Removing instances with no fo words less than a thresholf ie. 15

result = result[result['No words'].apply(lambda x: x > 15)]

In [ ]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40712 entries, 0 to 42203
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   W_ID      40712 non-null  int64 
 1   Plot      40712 non-null  object
 2   Genres    40712 non-null  object
 3   No words  40712 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.6+ MB


In [ ]:
# Extracting summary and genres for model training

x = result['Plot'].values
y = result['Genres'].values

In [ ]:
print(x[:4])

['shlykov a hard working taxi driver and lyosha a saxophonist develop a bizarre love hate relationship and despite their prejudices realize they aren t so different after all'
 'the nation of panem consists of a wealthy capitol and twelve poorer districts as punishment for a past rebellion each district must provide a boy and girl between the ages of and selected by lottery for the annual hunger games the tributes must fight to the death in an arena the sole survivor is rewarded with fame and wealth in her first reaping year old primrose everdeen is chosen from district her older sister katniss volunteers to take her place peeta mellark a baker s son who once gave katniss bread when she was starving is the other district tribute katniss and peeta are taken to the capitol accompanied by their frequently drunk mentor past victor haymitch abernathy he warns them about the career tributes who train intensively at special academies and almost always win during a tv interview with caesar fli

In [ ]:
print(y[:4])

[list(['drama']) list(['sci-fi', 'adventure', 'action', 'drama'])
 list(['action', 'musical', 'drama']) list(['comedy', 'screwball comedy'])]


In [ ]:
# Transforming Genres into multilabel binarizer

multi_ban = MultiLabelBinarizer()
multi_ban.fit(y)

# transform target variable
y = multi_ban.transform(y)

In [ ]:
multi_ban.inverse_transform(y[:1])

[('drama',)]

In [ ]:
num_classes = len(multi_ban.classes_)

In [ ]:
# set(multi_ban.classes_)

In [ ]:
# Splitting dataset into train and test

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 24)

In [ ]:
print(x_train.shape, y_test.shape)

(32569,) (8143, 100)


Importing Bert Tokenizer

In [ ]:
bert_tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
def encode_data(data, max_length):
  """ Encode the data according to required bert tokenization
      Args:
        data: text to be encoded
        max_length: The max length of text to be considered
  """
  encoded = bert_tokenizer.encode_plus(data,
                                  add_special_tokens=True,
                                  max_length=max_length,
                                  padding='max_length',
                                  truncation=True,
                                        pad_to_max_length=True,
                                  return_attention_mask=True)

  return encoded

In [ ]:
def encode_examples(x, y, max_length):
  """ Encodes data one by one using the encode data function and return a tuple
      containg the required tokens, mask and token type for bert input.
      Args:
        x: List containg movie summaries
        y: Genre info
        max_length: The max length of summary to be considered for tokenization
  """
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  label_list = []

  for review, label in zip(x, y):

    bert_input = encode_data(review, max_length)

    input_ids_list.append(bert_input['input_ids'])
    token_type_ids_list.append(bert_input['token_type_ids'])
    attention_mask_list.append(bert_input['attention_mask'])
    label_list.append(label)
  return (np.array(input_ids_list, dtype=np.float32), np.array(attention_mask_list, dtype=np.float32), np.array(token_type_ids_list, dtype=np.float32), np.array(label_list, dtype=np.float32))

In [ ]:
lr = 2e-5
epcohs = 1
max_length = 512
batch_size = 8

In [ ]:
# Preparing data for bert input

train_inp, train_mask, train_token_type, train_label  = encode_examples(x_train, y_train, max_length)

test_inp, test_mask, test_token_type, test_label = encode_examples(x_test, y_test, max_length)

In [ ]:
train_inp.shape

(32569, 512)

Importing the pre-trained bert model

In [ ]:

from transformers import TFBertModel

# model initialization
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
def create_model(bert_model):
  """ Adds a classification head of the bare bert model.
      Args:
        bert_model: pretrained bert model without classification head
  """
  input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32)
  token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32)
  attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32)

  input_dict = {
      'input_ids': input_ids,
      'attention_mask': attention_mask,
      'token_type_ids': token_type_ids
  }

  embedding = bert_model(input_dict)[0]
  avg = tf.keras.layers.GlobalAveragePooling1D()(embedding)
  drop = tf.keras.layers.Dropout(0.15)(avg)
  dense = tf.keras.layers.Dense(num_classes, activation='sigmoid')(drop)

  model = tf.keras.Model(
        inputs=[input_ids, attention_mask, token_type_ids],
        outputs=dense,
    )
  return model


In [ ]:
model = create_model(bert_model)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 512, 768), ( 109482240   input_3[0][0]                    
                                                                 input_1[0][0]         

In [ ]:
# optimizer Adam recommended
optimizer = tf.keras.optimizers.Adam(learning_rate=lr, epsilon=1e-08)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=tf.keras.metrics.Precision())

In [ ]:
train_inp.shape

(32569, 512)

In [ ]:
bert_history = model.fit([train_inp, train_mask, train_token_type], train_label, epochs=2, batch_size=batch_size, validation_data=(([test_inp, test_mask, test_token_type]), test_label))

Epoch 1/2
4072/4072 [==============================] - 2472s 607ms/step - loss: 0.0825 - precision: 0.6261 - val_loss: 0.0669 - val_precision: 0.6594
Epoch 2/2
4072/4072 [==============================] - 2469s 606ms/step - loss: 0.0617 - precision: 0.7113 - val_loss: 0.0635 - val_precision: 0.6811


In [ ]:
bert_history = model.fit([train_inp, train_mask, train_token_type], train_label, epochs=1, batch_size=batch_size, validation_data=(([test_inp, test_mask, test_token_type]), test_label))

4072/4072 [==============================] - 2469s 606ms/step - loss: 0.0535 - precision: 0.7520 - val_loss: 0.0645 - val_precision: 0.6614


In [ ]:
res = model.predict([test_inp[:25], test_mask[:25], test_token_type[:25]])

In [ ]:
res.shape

(25, 100)

In [ ]:
for a, b in zip(y_test[:15], res):
  print("Next instance")
  print()
  for i,j in zip(a,b):
    if i == 1:
      print(i, j)

Next instance

1 0.44887653
1 0.12331969
1 0.19497326
1 0.16118614
1 0.04585748
1 0.115512446
1 0.017563302
1 0.62020403
Next instance

1 0.5203328
1 0.7815401
1 0.21836558
1 0.29585883
1 0.93964696
1 0.08230562
Next instance

1 0.0037609856
1 0.6747603
Next instance

1 0.75570077
1 0.12806684
Next instance

1 0.01132794
1 0.0531769
1 0.03700808
1 0.90355587
1 0.010387511
1 0.011466391
1 0.18289644
Next instance

1 0.61062497
Next instance

1 0.39662808
1 0.22936088
1 0.044702135
1 0.9156513
Next instance

1 0.940624
1 0.16569397
1 0.021983238
Next instance

1 0.27199912
Next instance

1 0.714854
1 0.26899344
Next instance

1 0.9584871
1 0.49431136
Next instance

1 0.035866823
1 0.67000496
1 0.013366072
1 0.002022956
1 0.022406
Next instance

1 0.62423664
1 0.7193858
1 0.9332376
1 0.8130348
Next instance

1 0.59064627
1 0.466505
Next instance

1 0.21255527
1 0.32002512
1 0.052486


In [ ]:
a = np.argsort(res)[:, -5:]

In [ ]:
# Encoded classes of multi banarizer

inverse_list = multi_ban.classes_

In [ ]:
inverse_list[a]

array([['fiction', 'crime', 'drama', 'action', 'thriller'],
       ['costume', 'drama', 'action', 'adventure', 'fantasy'],
       ['adventure', 'fantasy', 'romance', 'drama', 'comedy'],
       ['thriller', 'fantasy', 'comedy', 'romance', 'drama'],
       ['culture', 'comedy', 'thriller', 'b-movie', 'horror'],
       ['adventure', 'fantasy', 'comedy', 'anime', 'family'],
       ['black-and-white', 'adventure', 'comedy', 'action', 'western'],
       ['action', 'romance', 'musical', 'anime', 'comedy'],
       ['comedy', 'action', 'drama', 'horror', 'thriller'],
       ['romance', 'slapstick', 'musical', 'black-and-white', 'comedy'],
       ['family', 'romance', 'musical', 'black-and-white', 'comedy'],
       ['family', 'coming of age', 'romance', 'drama', 'comedy'],
       ['adventure', 'childhood', 'fantasy', 'anime', 'family'],
       ['mystery', 'drama', 'comedy', 'thriller', 'horror'],
       ['musical', 'black-and-white', 'comedy', 'drama', 'romance'],
       ['family', 'anime', 'fan

In [ ]:
# Printing the actual output along with predicted by model

for actual, pred in zip(y_test[:25], a):
  print("Actual")
  print(multi_ban.inverse_transform(np.array([actual])))
  print("Predictions")
  pred = np.flip(pred)
  print(pred)
  print(inverse_list[pred])
  print()
  print("*******************")
  print()

Actual
[('action', 'adventure', 'crime', 'fiction', 'political', 'spy', 'suspense', 'thriller')]
Predictions
[94  1 34 28 44]
['thriller' 'action' 'drama' 'crime' 'fiction']

*******************

Actual
[('action', 'adventure', 'costume', 'drama', 'fantasy', 'sword and sorcery')]
Predictions
[42  3  1 34 26]
['fantasy' 'adventure' 'action' 'drama' 'costume']

*******************

Actual
[('documentary', 'drama')]
Predictions
[24 34 74 42  3]
['comedy' 'drama' 'romance' 'fantasy' 'adventure']

*******************

Actual
[('drama', 'fantasy')]
Predictions
[34 74 24 42 94]
['drama' 'romance' 'comedy' 'fantasy' 'thriller']

*******************

Actual
[('adventure', 'crime', 'fiction', 'horror', 'supernatural', 'teen', 'thriller')]
Predictions
[55 12 94 24 29]
['horror' 'b-movie' 'thriller' 'comedy' 'culture']

*******************

Actual
[('fantasy',)]
Predictions
[41  7 24 42  3]
['family' 'anime' 'comedy' 'fantasy' 'adventure']

*******************

Actual
[('action', 'adventure', 'dra

In [ ]:
# Mounting drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Saving weights on drive

model.save_weights('drive/My Drive/bert_wieghts.h5')

In [ ]:
# Checking if loading working properly by creating a new model
new_model = create_model(bert_model)

In [ ]:
new_model.load_weights('drive/My Drive/bert_wieghts.h5')

In [ ]:
res = new_model.predict([test_inp[:25], test_mask[:25], test_token_type[:25]])

In [ ]:
for actual, pred in zip(y_test[:25], a):
  print("Actual")
  print(multi_ban.inverse_transform(np.array([actual])))
  print("Predictions")
  pred = np.flip(pred)
  print(pred)
  print(inverse_list[pred])
  print()
  print("*******************")
  print()

Actual
[('action', 'adventure', 'crime', 'fiction', 'political', 'spy', 'suspense', 'thriller')]
Predictions
[94  1 34 28 44]
['thriller' 'action' 'drama' 'crime' 'fiction']

*******************

Actual
[('action', 'adventure', 'costume', 'drama', 'fantasy', 'sword and sorcery')]
Predictions
[42  3  1 34 26]
['fantasy' 'adventure' 'action' 'drama' 'costume']

*******************

Actual
[('documentary', 'drama')]
Predictions
[24 34 74 42  3]
['comedy' 'drama' 'romance' 'fantasy' 'adventure']

*******************

Actual
[('drama', 'fantasy')]
Predictions
[34 74 24 42 94]
['drama' 'romance' 'comedy' 'fantasy' 'thriller']

*******************

Actual
[('adventure', 'crime', 'fiction', 'horror', 'supernatural', 'teen', 'thriller')]
Predictions
[55 12 94 24 29]
['horror' 'b-movie' 'thriller' 'comedy' 'culture']

*******************

Actual
[('fantasy',)]
Predictions
[41  7 24 42  3]
['family' 'anime' 'comedy' 'fantasy' 'adventure']

*******************

Actual
[('action', 'adventure', 'dra